<a href="https://colab.research.google.com/github/Niccocracken/Niccocracken/blob/main/Herramienta_de_cortado_y_transcripcion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

# Importa las bibliotecas
from pydub import AudioSegment
import warnings
import whisper
import re
import os

# Carga el modelo de Whisper
model = whisper.load_model("small")

# Desactiva los mensajes de advertencia
warnings.simplefilter("ignore")

In [ ]:
# Define la ruta del audio
audio_ruta = input("Ruta del archivo: ")

# Carpeta donde se guardan los archivos de audio segmentados y transcripciones
carpeta_salida = "/content/transcripciones/"

# Se asegura de que la carpeta exista
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)

# Carga el audio
try:
    audio = AudioSegment.from_file(audio_ruta)
except Exception as e:
    print(f"Error cargando el audio: {e}")
    exit() # Sale si el audio no pudo ser cargado

duracion = len(audio)  # Duracion total del audio en milisegundos
segmento_duracion = 5 * 53000  # Duracion de cada segmento de audio en milisegundos (15 minutes)

transcripciones = []

# Corta y transcribe
for i in range(0, duracion, segmento_duracion):
    tiempo_inicio = i
    tiempo_final = min(i + segmento_duracion, duracion)  # Asegura que el último segmento no exceda el total del audio

    # Corta el audio
    audio_segmento = audio[tiempo_inicio:tiempo_final]

    # Define el nombre del archivo para cada segmento
    segmento_indice = i // segmento_duracion + 1
    segmento_nombre = os.path.join(carpeta_salida, f"audio_segmento_{segmento_indice}.mp3")
    transcripcion_nombre = os.path.join(carpeta_salida, f"transcripcion_audio_{segmento_indice}.txt")

    # Exporta el segmento de audio
    try:
        audio_segmento.export(segmento_nombre, format="mp3")
        print(f"Exportado {segmento_nombre}")
    except Exception as e:
        print(f"Error exportando el audio: {segmento_nombre}: {e}")
        continue # Salta al siguiente segmento si falla la exportación

    # Transcribe el segmmento de audio
    if os.path.exists(segmento_nombre):
        print(f"Transcribiendo {segmento_nombre}...")
        try:
            resultado = model.transcribe(segmento_nombre)
            transcripcion_texto = resultado["text"]

            # Separa párrafos de acuerdo al punto
            transcripcion_texto_formateado = re.sub(r'(?<!\d)(?<!\.)\.(?!\.)', '.\n\n', transcripcion_texto)

            # Elimina los espacios iniciales de cada línea
            transcripcion_texto_formateado = re.sub(r'^[ \t]+(?=\S)', '', transcripcion_texto_formateado, flags=re.MULTILINE)

            # Guarda la transcripcion en un .txt
            with open(transcripcion_nombre, "w") as f:
                f.write(transcripcion_texto_formateado)

            print(f"Transcripcion guardada en {transcripcion_nombre}")
            transcripciones.append((segmento_indice, transcripcion_texto_formateado)) # Almacena transcripcion con su indice
        except Exception as e:
            print(f"Error transcribiendo {segmento_nombre}: {e}")
    else:
        print(f"Segmento no encontrado: {segmento_nombre}")

print("Proceso de cortado y transcripcion completado.")

# Organiza las transcripciones por su indice
transcripciones.sort(key=lambda item: item[0])

# Combina todas las transcripciones en un solo archivo
archivo_combinado_salida = os.path.join(carpeta_salida, "transcripcion_audio_completa.txt")

with open(archivo_combinado_salida, 'w') as archivo_salida:
    for index, transcripcion in transcripciones:
        archivo_salida.write(transcripcion)

print(f"Todas las transcripciones han sido combinadas en {archivo_combinado_salida}")
